# Playground Error Linier Interaktif

Gunakan kontrol di samping untuk menggeser posisi tiga titik data dan menyesuaikan garis referensi. Visualisasi akan langsung memperlihatkan garis vertikal (selisih `y`) dan nilai error setiap titik.

**Cara pakai:** Geser slider `x` dan `y` untuk tiap titik, lalu atur `slope` serta `intercept` garis referensi. Grafik dan ringkasan error akan ikut berubah secara dinamis sehingga memudahkan eksplorasi konsep residual pada regresi linier.

In [1]:
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output

# Slider untuk koordinat titik dan parameter garis
x1_slider = widgets.FloatSlider(value=1.0, min=-10.0, max=10.0, step=0.5, description='x1')
y1_slider = widgets.FloatSlider(value=2.5, min=-10.0, max=10.0, step=0.5, description='y1')
x2_slider = widgets.FloatSlider(value=3.0, min=-10.0, max=10.0, step=0.5, description='x2')
y2_slider = widgets.FloatSlider(value=3.8, min=-10.0, max=10.0, step=0.5, description='y2')
x3_slider = widgets.FloatSlider(value=5.0, min=-10.0, max=10.0, step=0.5, description='x3')
y3_slider = widgets.FloatSlider(value=4.1, min=-10.0, max=10.0, step=0.5, description='y3')
slope_slider = widgets.FloatSlider(value=0.3, min=-3.0, max=3.0, step=0.1, description='slope')
intercept_slider = widgets.FloatSlider(value=2.5, min=-10.0, max=10.0, step=0.5, description='intercept')

sliders = [
    x1_slider, y1_slider, x2_slider, y2_slider, x3_slider, y3_slider, slope_slider, intercept_slider
]

# Tata letak kontrol agar ringkas dibaca
controls_layout = widgets.Layout(
    display='grid',
    grid_template_columns='repeat(2, minmax(160px, 1fr))',
    grid_gap='8px',
    padding='4px'
)
controls_box = widgets.GridBox(children=sliders, layout=controls_layout)

out = widgets.Output()

def update_plot(change=None):
    with out:
        clear_output(wait=True)
        x_coords = [x1_slider.value, x2_slider.value, x3_slider.value]
        y_coords = [y1_slider.value, y2_slider.value, y3_slider.value]
        slope = slope_slider.value
        intercept = intercept_slider.value

        x_span = max(x_coords) - min(x_coords)
        padding = max(1.5, 0.2 * x_span)
        x_min = min(x_coords) - padding
        x_max = max(x_coords) + padding
        xs = np.linspace(x_min, x_max, 200)
        line_vals = slope * xs + intercept

        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=x_coords,
            y=y_coords,
            mode='markers',
            marker=dict(size=12, color='royalblue'),
            name='Titik data'
))
        fig.add_trace(go.Scatter(
            x=xs,
            y=line_vals,
            mode='lines',
            line=dict(color='darkorange'),
            name='Garis referensi'
))

        error_rows = []
        for idx, (x, y) in enumerate(zip(x_coords, y_coords), start=1):
            y_on_line = slope * x + intercept
            error = y - y_on_line
            fig.add_trace(go.Scatter(
                x=[x, x],
                y=[y_on_line, y],
                mode='lines',
                line=dict(color='gray', dash='dash'),
                showlegend=False
))
            fig.add_annotation(
                x=x,
                y=(y + y_on_line) / 2,
                text=f'Δ={error:.2f}',
                showarrow=False,
                font=dict(size=10, color='black'),
                bgcolor='rgba(255,255,255,0.7)'
)
            error_rows.append((idx, x, y, y_on_line, error, abs(error), error ** 2))

        fig.update_layout(
            title='Residual vertikal terhadap garis linier',
            xaxis_title='x',
            yaxis_title='y',
            legend=dict(x=0.02, y=0.98),
            margin=dict(l=40, r=20, t=50, b=40),
            height=420
)

        total_abs = sum(row[5] for row in error_rows)
        total_sq = sum(row[6] for row in error_rows)

        summary_lines = [
            '<b>Ringkasan Error</b>',
            f'Total |Error| = {total_abs:.3f}',
            f'Total Error^2 = {total_sq:.4f}'
]
        detail_lines = [
            "<hr style='margin:6px 0;'>",
            '<b>Per titik:</b>'
]
        for idx, x, y, y_hat, err, abs_err, sq_err in error_rows:
            detail_lines.append(
                f'Titik {idx}: x={x:.2f}, y={y:.2f}, y_garis={y_hat:.2f}, Δ={err:.2f}, |Δ|={abs_err:.2f}, Δ²={sq_err:.4f}'
)

        fig.show()
        summary_html = '<br>'.join(summary_lines + detail_lines)
        display(widgets.HTML(summary_html))

for slider in sliders:
    slider.observe(update_plot, names='value')

update_plot()

layout = widgets.Layout(align_items='flex-start')
display(widgets.HBox([controls_box, out], layout=layout))